"""
AI Agent Automation with LangGraph - Complete Implementation
Customer Support Automation System using Groq API
"""

In [2]:
import os
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
import operator
from datetime import datetime
import json
import logging

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)

In [6]:
# ============================================================================
# STATE DEFINITION
# ============================================================================

class AgentState(TypedDict):
    query: str
    category: str
    retrieved_data: str
    response: str
    quality_score: int
    timestamp: str
    metadata: dict

In [7]:
# ============================================================================
# CONFIGURATION
# ============================================================================
class Config:
    """Configuration for AI Agent System"""
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    MODEL_NAME = "llama-3.3-70b-versatile"
    TEMPERATURE = 0.7
    MAX_TOKENS = 1000
    
    #Knowledge Base Simulation
    KNOWLEDGE_BASE = {
        'ORDER_STATUS': {
            "data": "Order tracking system connected. Typical delivery: 3-5 business days.",
            "context": "Customer inquiring about order status"
        },
        'REFUND': {
            "data": "Refund policy: 30-day return window. Full refund with receipt. Processing time: 5-7 business days.",
            "context": "Customer requesting refund information"
        },
        'SHIPPING': {
            "data": "Shipping options: Standard (5-7 days, $5.99), Express (2-3 days, $12.99), Overnight ($24.99).",
            "context": "Customer inquiring about shipping"
        },
        'POLICY': {
            "data": "Return policy: 30 days, original condition. Warranty: 1 year manufacturer warranty on electronics.",
            "context": "Customer asking about policies"
        },
        'TECHNICAL_SUPPORT': {
            "data": "Technical support available 24/7. Common issues: password reset, account access, product setup.",
            "context": "Customer needs technical assistance"
        },
        'GENERAL': {
            "data": "General customer support. Business hours: Mon-Fri 9AM-6PM EST. Contact: support@company.com",
            "context": "General inquiry"
        }
    }

In [8]:
# ============================================================================
# LLM INITIALIZATION
# ============================================================================
def initialize_llm():
    """Initialize the Groq LLM"""
    try:
        llm = ChatGroq(
            model=Config.MODEL_NAME,
            temperature=Config.TEMPERATURE,
            max_tokens=Config.MAX_TOKENS,
            groq_api_key=Config.GROQ_API_KEY
        )
        logger.info("LLM initialized successfully")
        return llm
    except Exception as e:
        logger.error(f"Failed to initialized LLM: {e}")
        raise

In [9]:
# ============================================================================
# AGENT NODES
# ============================================================================
def classify_query_node(state: AgentState) -> AgentState:
    """
    Node 1: Classify the incoming query into categories
    """
    logger.info(f"Classifying query: {state['query']}")
    
    try:
        llm = initialize_llm()
        
        prompt = f"""Analyze this customer support query and classify it into ONE of this categories:
        - ORDER_STATUS: Questions about order tracking, delivery status
        - REFUND: Refund requests, return inquiries
        - SHIPPING: Shipping methods, costs, delivery times
        - POLICY: Return policies, warranties, terms
        - TECHNICAL_SUPPORT: Technical issues, troubleshooting
        - GENERAL: Everything else
        
        Query: {state['query']}
        
        Responde with ONLY the category name, nothing else."""
        
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)
        category = response.content.strip()
        
        logger.info(f"Query classified as: {category}")
        
        return {
            **state,
            "category": category,
            "timestamp": datetime.now().isoformat(),
            "metadata": {
                "node": "classifier",
                "status": "completed"
            }
        }
    except Exception as e:
        logger.error(f"Error in classify_query_node: {e}")
        return {
            **state,
            "category": "GENERAL",
            "metadata": {
                "node": "classifier",
                "status": "error",
                "error": str(e)
            }
        }

In [10]:
def retrieve_data_node(state: AgentState) -> AgentState:
    """
    Node 2: Retrieve relevant data based on classification
    """
    logger.info(f"Retrieving data for category: {state['category']}")
    
    try:
        category = state.get('category', "GENERAL")
        
        # Retrieve from knowledge base
        knowledge = Config.KNOWLEDGE_BASE.get(category, Config.KNOWLEDGE_BASE["GENERAL"])
        retrieved_data = knowledge["data"]
        
        logger.info(f"Data retrieved successfully for {category}")
        
        # Return updated state
        return {
            **state, # Unpack the previous state
            "retrieved_data": retrieved_data,
            "metadata": {
                **state.get("metadata", {}),
                "retriever": "completed",
                "data_source": "knowledge_base"
            }
        }
    except Exception as e:
        logger.error(f"Error retrieved_data_node: {e}")
        return {
            **state, 
            "retrieved_data": "Error retrieving data",
            "metadata": {
                **state.get("metadata", {}),
                "retriever": "error"
            }
        }